# Метод ближайших соседей (бинарная классификация)

Метод ближайших соседей (k Nearest Neighbors, или kNN) — простой алгоритм классификации (регрессии), основанный на оценивании сходства объектов. Суть метода проста: объект относится к тому классу, к которому принадлежит большинство из его ближайших k соседей. В данном классе реализованы классический и взвешенный KNN.

Реализованы методы:
- fit для обучения модели
- predict для предсказания таргетов
- _init_ - конструктор
- predict_proba - возвращает вероятности для класса 1
- calculate_metric - подсчитывает нужные метрики
- predict_with_weight - возвращает предсказание класса или вероятность для класса 1 в зависимсости от weight


In [1]:
import pandas as pd
import numpy as np

In [2]:
class MyKNNClf():
    
    def __init__(self, k=3, metric='euclidean', weight='uniform'):
        '''
        Input:
        int k: the number of nearest neighbors that we will consider when defining the class. (default = 3)
        str metric: name of the metric from array ['euclidean', 'chebyshev', 'manhattan', 'cosine'] 
                                                                                        (default = 'euclidean')
        str weight: name of the metod for weighted kNN from array ['uniform', 'rank', 'distance'] 
                                                                                        (default = 'uniform')
        '''
        self.k = k
        self.metric = metric
        self.weight = weight
        # size of the training sample
        self.train_size = None
        self.X = None
        self.y = None

    
    def __str__(self):
        '''
        Output: 
        string - info about class parameters
        '''
        return f"MyKNNClf class: k={self.k}"
    
    def fit(self, X, y):
        '''
        Input:
        pd.DataFrame X: features
        pd.Series y: targets
        '''
        self.X = X
        self.y = y
        self.train_size = X.shape
        
    def predict(self, X):
        '''
        Input:
        pd.DataFrame X: features
        Output:
        np.array of predictions
        '''
        y_pred = []
        for i in range(X.shape[0]):
            distances = self.calculate_metric(X_test.iloc[i])
            distances = distances.sort_values()
            nearest_indices = distances.head(self.k).index
            nearest_classes = self.y.loc[nearest_indices]
            y_pred.append(self.predict_with_weight(nearest_classes, distances))
        return np.array(y_pred)
    
    def predict_proba(self, X):
        '''
        Input:
        pd.DataFrame X: features
        Output:
        np.array of probabilities
        '''
        y_proba = []
        for i in range(X.shape[0]):
            distances = self.calculate_metric(X_test.iloc[i])
            distances = distances.sort_values()
            nearest_indices = distances.head(self.k).index
            nearest_classes = self.y.loc[nearest_indices]
            y_proba.append(self.predict_with_weight(nearest_classes, distances, is_proba=True))
        return np.array(y_proba)
    
    def predict_with_weight(self, nearest_classes, distances, is_proba=False):
        '''
        Input:
        nearest_classes: array of classifications
        distances: values of metric
        Output:
        classification or probability for class 1
        '''
        if self.weight == 'uniform':
            if is_proba:
                return sum(nearest_classes) / self.k
            
            return 1 if nearest_classes.sum() >= self.k / 2 else 0
        
        elif self.weight == 'rank':
            nearest_classes = nearest_classes.reset_index()
            del nearest_classes['index']
            class_0 = sum(1 / (nearest_classes[nearest_classes.iloc[:, 0] == 0].index + 1)) / sum(1 / (nearest_classes.index + 1))
            class_1 = sum(1 / (nearest_classes[nearest_classes.iloc[:, 0] == 1].index + 1)) / sum(1 / (nearest_classes.index + 1))
            if is_proba:
                return class_1
            
            return 1 if class_1 >= class_0 else 0
        
        elif self.weight == 'distance':
            class_0 = sum(1 / (distances.loc[nearest_classes[nearest_classes == 0].index])) / sum(1 / (distances.loc[nearest_classes.index]))
            class_1 = sum(1 / (distances.loc[nearest_classes[nearest_classes == 1].index])) / sum(1 / (distances.loc[nearest_classes.index]))
            if is_proba:
                return class_1
            
            return 1 if class_1 >= class_0 else 0
                
        
    def calculate_metric(self, x):
        '''
        Input:
        pd.Series x: vector
        Output:
        pd.Series with values of metric 
        '''
        if self.metric == 'euclidean':
            return np.sqrt(((self.X - x) ** 2).sum(axis=1))
        elif self.metric == 'chebyshev':
            return np.abs(self.X - x).max(axis=1)
        elif self.metric == 'manhattan':
            return np.abs(self.X - x).sum(axis=1)
        elif self.metric == 'cosine':
            dot_product = (self.X * x).sum(axis=1)
            norms = np.linalg.norm(self.X, axis=1) * np.linalg.norm(x)
            return 1 - dot_product / norms

## Протестируем модель

Входные данные: датасет с различными параметрами

Выходные данные: возвращенные предсказания


In [3]:
df = pd.read_csv('../data_banknote_authentication.txt', header=None)
df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'target']
X, y = df.iloc[:,:4], df['target']

In [4]:
from sklearn.model_selection import train_test_split

model = MyKNNClf(5, 'cosine', 'rank')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
np.sum(y_pred - y_test)

-1

Как ожидалось, наша модель выдает маленькую ошибку. Следовательно, алгоритм реализован верно.